In [1]:
from median_approx import median_approximation
from tools import build_graphs_from_file, relabel_graph_to_mutag, show_graph
from lib.ot_distances import Fused_Gromov_Wasserstein_distance

mutag_labels = ["C", "O", "Cl", "H", "N", "F", "Br", "S", "P", "I", "Na", "K", "Li", "Ca"]

In [2]:
def graph_distance(graph1, graph2, alpha=0.9):
    return Fused_Gromov_Wasserstein_distance(alpha=alpha, features_metric='dirac', method='shortest_path').graph_d(
        graph1, graph2)

In [3]:
rule = "23"
cls = 0
alpha = 0.90
a = "90"

In [4]:
path_to_active_egos = "../activ_ego/mutag_" + rule + "labels_egos.txt"
graphs_cls, means_number_of_nodes = build_graphs_from_file(path_to_active_egos)
graphs = graphs_cls[cls]
len(graphs)

Mean size of graphs: [9.14756447 9.59137056]


3490

In [5]:
def show_graph(g, save = False):
        show_graph(relabel_graph_to_mutag(g, mutag_labels), name="median_graph", layout="kamada_kawai", title="median graph", save=save)

In [ ]:
show_graph(median_approximation(graphs, alpha=0.9, t=10E-10))

/home/elouan/epita/lrde/optimal_transport_for_gnn/src/lib/graph.py:145: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(self.nx_graph)
